Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Onnx Pipeline

This repository shows how to deploy and use Onnx pipeline with dockers including convert model, generate input and performance test.

# Prerequisites

Pull dockers from Azure. It should take several minutes.

In [1]:
!sh build.sh

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Error response from daemon: Get https://ziylregistry.azurecr.io/v2/: unauthorized: authentication required
Using default tag: latest
latest: Pulling from onnx-converter
Digest: sha256:59366705ce552225e0e18e17dbc353d81414809c4a1d00c9ad5fc04674a49537
Status: Image is up to date for ziylregistry.azurecr.io/onnx-converter:latest
Using default tag: latest
latest: Pulling from perf-test
Digest: sha256:22e14856e1dde45b28b5b7f013286064858e5708e677a22bd4d40325c269c300
Status: Image is up to date for ziylregistry.azurecr.io/perf-test:latest


Install the onnxpipeline SDK

In [2]:
import onnxpipeline

pipeline = onnxpipeline.Pipeline()

# onnx
#pipeline = onnxpipeline.Pipeline('onnx')

# tensorflow
#pipeline = onnxpipeline.Pipeline('mnist/model')

# pytorch
#pipeline = onnxpipeline.Pipeline('pytorch')

# cntk 
#pipeline = onnxpipeline.Pipeline('cntk')

# keras
#pipeline = onnxpipeline.Pipeline('KerasToONNX')

# sklearn
#pipeline = onnxpipeline.Pipeline('sklearn')

# caffe
#pipeline = onnxpipeline.Pipeline('caffe')

# current directory
#pipeline = onnxpipeline.Pipeline()

# test mxnet fail
#pipeline = onnxpipeline.Pipeline('mxnet')


## Run parameters

(1) local_directory: string

    Required. The path of local directory where would be mounted to the docker. All operations will be executed from this path.

(2) mount_path: string

    Optional. The path where the local_directory will be mounted in the docker. Default is "/mnt/model".

(3) print_logs: boolean

    Optional. Whether print the logs from the docker. Default is True.
    
(4) convert_directory: string

    Optional. The directory path for converting model. Default is test/.    

(5) convert_name: string

    Optional. The model name for converting model. Default is model.onnx.
    


# Config information for ONNX pipeline

In [3]:
pipeline.config()

-----------config----------------
           Container information: <docker.client.DockerClient object at 0x7f1913d70c50>
 Local directory path for volume: /home/chuche/notebook
Volume directory path in dockers: /mnt/model
                     Result path: result
        Converted directory path: test
        Converted model filename: model.onnx
            Converted model path: test/model.onnx
        Print logs in the docker: True


# Convert model to ONNX

This image is used to convert model from major model frameworks to onnx. Supported frameworks are - caffe, cntk, coreml, keras, libsvm, mxnet, scikit-learn, tensorflow and pytorch.


You can run the docker image with customized parameters.

In [4]:

model = pipeline.convert_model(model='onnx/perf-test/ImageQnA.onnx', model_type='onnx')

# test tensorflow
#model = pipeline.convert_model(model_type='tensorflow')

# test pytorch
#model = pipeline.convert_model(model_type='pytorch', model='saved_model.pb', model_input_shapes='(1,3,224,224)')

# test cntk
#model = pipeline.convert_model(model_type='cntk', model='ResNet50_ImageNet_Caffe.model')

# test keras
#model = pipeline.convert_model(model_type='keras', model='keras_Average_ImageNet.keras')

# test sklearn
#model = pipeline.convert_model(model_type='scikit-learn', model='sklearn_svc.joblib', initial_types=("float_input", "FloatTensorType([1,4])"))

# test caffe
#model = pipeline.convert_model(model_type='caffe', model='bvlc_alexnet.caffemodel')

# test mxnet
#model = pipeline.convert_model(model_type='mxnet', model='resnet.json', model_params='resnet.params', model_input_shapes='(1,3,224,224)')



-------------

Model Conversion



Input model is already ONNX model. Skipping conversion.



-------------

MODEL INPUT GENERATION(if needed)



Input.pb already exists. Skipping dummy input generation. 



-------------

MODEL CORRECTNESS VERIFICATION





Check the ONNX model for validity 

The ONNX model is valid.



Check ONNX model for correctness. 

The original model is already onnx. Skipping correctness test. 



-------------

MODEL CONVERSION SUMMARY (.json file generated at /mnt/model/test/output.json )



{'conversion_status': 'SUCCESS',

 'correctness_verified': 'SKIPPED',

 'error_message': '',

 'input_folder': '/mnt/model/test/test_data_set_0',

 'output_onnx_path': '/mnt/model/test/model.onnx'}



## Run parameters

(1) model: string

    Required. The path of the model that needs to be converted.

(2) output_onnx_path: string

    Required. The path to store the converted onnx model. Should end with ".onnx". e.g. output.onnx

(3) model_type: string

    Required. The name of original model framework. Available types are caffe, cntk, coreml, keras, libsvm, mxnet, scikit-learn, tensorflow and pytorch.

(4) model_inputs_names: string

    (tensorflow) Optional. The model's input names. Required for tensorflow frozen models and checkpoints.

(5) model_outputs_names: string

    (tensorflow) Optional. The model's output names. Required for tensorflow frozen models checkpoints.

(6) model_params: string 

    (mxnet) Optional. The params of the model if needed.

(7) model_input_shapes: list of tuple 

    (pytorch, mxnet) Optional. List of tuples. The input shape(s) of the model. Each dimension separated by ','.

(8) target_opset: int

    Optional. Specifies the opset for ONNX, for example, 7 for ONNX 1.2, and 8 for ONNX 1.3. Defaults to 7.
    
(9) caffe_model_prototxt: string

    (caffe) Optional. The filename of deploy prototxt for the caffe madel. 

(10) initial_types: tuple (string, string)

    (scikit-learn) Optional. A tuple consist two strings. The first is data type and the second is the size of tensor type e.g., ('float_input', 'FloatTensorType([1,4])')

(11) input_json: string

    Optional. Use JSON file as input parameters.

(12) convert_json: boolean
    
    Optional. Convert all parameters into JSON file for input parameters.
    
(13) model_inputs_names: string

    (tensorflow) Optional.
    
(14) model_outputs_names: string

    (tensorflow) Optional.    


# Performance test tool

You can run perf_test using command python perf_test.py [Your model path] [Output path on the docker]. You can use the same arguments as for onnxruntime_pert_test tool, e.g. -m for mode, -e to specify execution provider etc. By default it will try all providers available.

In [5]:
result = pipeline.perf_test(model=model)
#result = pipeline.perf_test()   # is ok, too

Setting thread pool size to 1

Found unsupport node: 0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/SrcEncoder/BidirRnnEncoder/fw/fw/strided_slice__86:0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupport node: 0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-ten

Total time cost:0.0186173

Total iterations:20

Average time cost:0.930864 ms

Setting thread pool size to 4

Found unsupport node: 0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/SrcEncoder/BidirRnnEncoder/fw/fw/strided_slice__86:0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupport node: 0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_uti

Total time cost:0.0187323

Total iterations:20

Average time cost:0.936616 ms

Setting thread pool size to 2

Found unsupport node: 0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/SrcEncoder/BidirRnnEncoder/fw/fw/strided_slice__86:0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupport node: 0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_uti

Total time cost:0.0186672

Total iterations:20

Average time cost:0.933359 ms

Setting thread pool size to 3

Found unsupport node: 0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/SrcEncoder/BidirRnnEncoder/fw/fw/strided_slice__86:0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupport node: 0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_uti

Total time cost:0.0187922

Total iterations:20

Average time cost:0.939612 ms

Setting thread pool size to 2

Found unsupport node: 0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/SrcEncoder/BidirRnnEncoder/fw/fw/strided_slice__86:0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_utils.hpp

Found unsupport node: 0

Found unsupported input: /home/ziyl/onnxruntime/cmake/external/onnx-tensorrt/onnx2trt_uti

...

Setting thread pool size to 5

Total time cost:0.0183041

Total iterations:20

Average time cost:0.915204 ms

2019-07-01 23:32:52.250691669 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/TiedEmbedder/WordConvEmbedding/Reshape__72

2019-07-01 23:32:52.250740622 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/TiedEmbedder/WordConvEmbedding/Reshape_1__64

2019-07-01 23:32:52.250764600 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/TiedEmbedder/WordConvEmbedding/Word

Setting thread pool size to 6

Total time cost:0.0184998

Total iterations:20

Average time cost:0.924991 ms

Setting thread pool size to 0

Total time cost:0.0198857

Total iterations:20

Average time cost:0.994284 ms

Setting thread pool size to 0

Total time cost:0.019009

Total iterations:20

Average time cost:0.950452 ms

Setting thread pool size to 0

Total time cost:0.0156142

Total iterations:20

Average time cost:0.78071 ms

Setting thread pool size to 0

Total time cost:0.0195784

Total iterations:20

Average time cost:0.978918 ms

Setting thread pool size to 0

Total time cost:0.019843

Total iterations:20

Average time cost:0.99215 ms

Setting thread pool size to 0

Total time cost:0.0252664

Total iterations:20

Average time cost:1.26332 ms

Setting thread pool size to 0

Total time cost:0.028995

Total iterations:20

Average time cost:1.44975 ms

Setting thread pool size to 0

Total time cost:0.0198705

Total iterations:20

Average time cost:0.993527 ms

Setting thread po

/perf_test/bin/RelWithDebInfo/mkldnn_mklml/onnxruntime_perf_test -e mkldnn -x 4 -m times -r 20 /mnt/model/test/model.onnx /mnt/model/result/cc5d3e14-9480-4926-a82a-4e279b4ed31aSetting thread pool size to 2

Total time cost:0.0202147

Total iterations:20

Average time cost:1.01074 ms

Setting thread pool size to 1

Total time cost:0.0259925

Total iterations:20

Average time cost:1.29962 ms

Setting thread pool size to 4

Total time cost:0.0207077

Total iterations:20

Average time cost:1.03538 ms

Setting thread pool size to 5

Total time cost:0.0191979

Total iterations:20

Average time cost:0.959894 ms

Setting thread pool size to 6

Total time cost:0.018084

Total iterations:20

Average time cost:0.904202 ms

Setting thread pool size to 1

Total time cost:6.23968

Total iterations:20

Average time cost:311.984 ms

Setting thread pool size to 4

Total time cost:6.2273

Total iterations:20

Average time cost:311.365 ms

Setting thread pool size to 5

Total time cost:6.16801

Total ite

Total time cost:0.0184693

Total iterations:20

Average time cost:0.923463 ms

Setting thread pool size to 0

Total time cost:0.0197709

Total iterations:20

Average time cost:0.988544 ms

2019-07-01 23:33:50.761430724 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/TiedEmbedder/WordConvEmbedding/Reshape__72

2019-07-01 23:33:50.761477477 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spacev_co_train_norm_loss_HybridSequenceSimilarityModel_1_0_2/TiedEmbedder/WordConvEmbedding/Reshape_1__64

2019-07-01 23:33:50.761501338 [W:onnxruntime:Default, cuda_execution_provider.cc:1082 GetCapability] Fallback to CPU execution provider for Op type: Cast node name: agi.lib.tf.model_repository.spacev_co_train_no

Setting thread pool size to 0

Total time cost:0.0185768

Total iterations:20

Average time cost:0.928838 ms

Setting thread pool size to 0

Total time cost:0.0174943

Total iterations:20

Average time cost:0.874714 ms

Setting thread pool size to 0

Total time cost:0.0463579

Total iterations:20

Average time cost:2.31789 ms

Setting thread pool size to 0

Total time cost:0.0179751

Total iterations:20

Average time cost:0.898753 ms

Setting thread pool size to 0

Total time cost:14.524

Total iterations:20

Average time cost:726.198 ms

Setting thread pool size to 0

Total time cost:0.0571724

Total iterations:20

Average time cost:2.85862 ms

Setting thread pool size to 0

Total time cost:6.19814

Total iterations:20

Average time cost:309.907 ms

Setting thread pool size to 0

Total time cost:0.0223811

Total iterations:20

Average time cost:1.11905 ms

Setting thread pool size to 0

Total time cost:0.0184265

Total iterations:20

Average time cost:0.921326 ms

Setting thread pool 

# Run parameters

(1) model: string

    Required. The path of the model that wants to be performed.
    
(2) result: string

    Optional. The path of the result.
    
(3) config: string (choices=["Debug", "MinSizeRel", "Release", "RelWithDebInfo"])

    Optional. Configuration to run. Default is "RelWithDebInfo".
    
(4) m: string (choices=["duration", "times"])

    Optional. Specifies the test mode. Value could be 'duration' or 'times'. Default is "times".

(5) e: string (choices=["cpu", "cuda", "mkldnn"])

    Optional. help="Specifies the provider 'cpu','cuda','mkldnn'. Default is ''.
    
(6) r: string

    Optional. Specifies the repeated times if running in 'times' test mode. Default:20.
    
(7) t: string

    Optional. Specifies the seconds to run for 'duration' mode. Default:10.
    
(8) x: string

    Optional. Use parallel executor, default (without -x): sequential executor.
    
(9) n: string

    Optional. OMP_NUM_THREADS value.
    
(10) s: string

    Optional. Show percentiles for top n runs. Default:5.
    
(11) runtime: boolean

    Optional. Optional. Use this boolean flag to enable GPU if you have one.
   

In [6]:
pipeline.print_performance(result)
#pipeline.print_result() # is ok, too

cpu_openmp_6_threads_OMP_WAIT_POLICY_active 0.000589126 s

cpu_openmp_6_threads 0.000603389 s

mkldnn_openmp_5_threads 0.000651238 s

cpu_openmp_5_threads_OMP_WAIT_POLICY_active 0.000661868 s

cpu_openmp_5_threads 0.000667182 s

mkldnn_openmp_6_threads_OMP_WAIT_POLICY_active 0.000778366 s

mkldnn_openmp_5_threads_OMP_WAIT_POLICY_active 0.000782432 s

mkldnn_openmp_6_threads 0.000854094 s

cpu_openmp_OMP_WAIT_POLICY_active 0.000870979 s

cpu 0.000907307 s

mkldnn_openmp_4_threads 0.000915852 s

cuda_5_threads 0.000916611 s

cpu_openmp 0.000921363 s

cuda_6_threads 0.000923944 s

tensorrt 0.000927894 s

tensorrt_3_threads 0.000929308 s

tensorrt_2_threads 0.00093027 s

cuda_4_threads 0.000931572 s

tensorrt_1_threads 0.000932731 s

cpu_openmp_4_threads 0.000933936 s

tensorrt_4_threads 0.000934369 s

tensorrt_2_threads 0.000934712 s

cuda_1_threads 0.000935994 s

cpu_openmp_4_threads_OMP_WAIT_POLICY_active 0.00094483 s

cpu_5_threads 0.000948074 s

mkldnn_openmp_4_threads_OMP_WAIT_POLICY

# Result Visualization

After performance test, there would be a directory for results. 

This libray use Pandas.read_json to visualize JSON file. (orient is changeable.)

"latency.json" contains the raw data of results ordered by the average time. 

Use .latency to obtain the original latency JSON; Use .profiling to obtain original top 5 profiling JSON.

In [10]:
r = pipeline.get_result(result)
#r.latency
#r.profiling

[{u'avg': 0.000589126,
  u'code_snippet': {u'code': u'                import onnxruntime as ort                 so = rt.SessionOptions()                 so.set_graph_optimization_level(2)                 so.enable_sequential_execution = True                 so.session_thread_pool_size(0)                 session = rt.Session("/mnt/model/test/model.onnx", so)                 ',
   u'environment_variables': {u'LD_LIBRARY_PATH': u'/perf_test/bin/RelWithDebInfo/cpu_openmp',
    u'OMP_WAIT_POLICY': u'active'},
   u'execution_provider': u''},
  u'command': u'/perf_test/bin/RelWithDebInfo/cpu_openmp/onnxruntime_perf_test -m times -r 20 /mnt/model/test/model.onnx /mnt/model/result/80612513-e258-4036-994a-efa9a5034096',
  u'cpu_usage': 0.0,
  u'gpu_usage': 0,
  u'memory_util': 0.0,
  u'name': u'cpu_openmp_6_threads_OMP_WAIT_POLICY_active',
  u'p90': 0.000724701,
  u'p95': 0.000778397},
 {u'avg': 0.000603389,
  u'code_snippet': {u'code': u'                import onnxruntime as ort                

# Print latency.json

In [11]:
r.prints()

,avg,code_snippet,command,cpu_usage,gpu_usage,memory_util,name,p90,p95
0,0.000589,{u'environment_variables': {u'OMP_WAIT_POLICY'...,/perf_test/bin/RelWithDebInfo/cpu_openmp/onnxr...,0,0,0,cpu_openmp_6_threads_OMP_WAIT_POLICY_active,0.000725,0.000778
1,0.000603,{u'environment_variables': {u'LD_LIBRARY_PATH'...,/perf_test/bin/RelWithDebInfo/cpu_openmp/onnxr...,0,0,0,cpu_openmp_6_threads,0.000784,0.000790
2,0.000651,{u'environment_variables': {u'LD_LIBRARY_PATH'...,/perf_test/bin/RelWithDebInfo/mkldnn_openmp/on...,0,0,0,mkldnn_openmp_5_threads,0.000699,0.000774
3,0.000662,{u'environment_variables': {u'OMP_WAIT_POLICY'...,/perf_test/bin/RelWithDebInfo/cpu_openmp/onnxr...,0,0,0,cpu_openmp_5_threads_OMP_WAIT_POLICY_active,0.000724,0.000777
4,0.000667,{u'environment_variables': {u'LD_LIBRARY_PATH'...,/perf_test/bin/RelWithDebInfo/cpu_openmp/onnxr...,0,0,0,cpu_openmp_5_threads,0.000692,0.000787
5,0.000778,{u'environment_variables': {u'OMP_WAIT_POLICY'...,/perf_test/bin/RelWithDebInfo/mkldnn_openmp/on...,0,0,0,mkldnn_openmp_6_threads_OMP_WAIT_POLICY_active,0.000834,0.000868
6,0.000782,{u'environment_variables': {u'OMP_WAIT_POLICY'...,/perf_test/bin/RelWithDebInfo/mkldnn_openmp/on...,0,0,0,mkldnn_openmp_5_threads_OMP_WAIT_POLICY_active,0.000803,0.000840
7,0.000854,{u'environment_variables': {u'LD_LIBRARY_PATH'...,/perf_test/bin/RelWithDebInfo/mkldnn_openmp/on...,0,0,0,mkldnn_openmp_6_threads,0.000933,0.000981
8,0.000871,{u'environment_variables': {u'OMP_WAIT_POLICY'...,/perf_test/bin/RelWithDebInfo/cpu_openmp/onnxr...,0,0,0,cpu_openmp_OMP_WAIT_POLICY_active,0.000984,0.000999
9,0.000907,{u'environment_variables': {u'LD_LIBRARY_PATH'...,/perf_test/bin/RelWithDebInfo/cpu/onnxruntime_...,0,0,0,cpu,0.000989,0.000991


# Print profiling.json

Only provide profiling JSON for top 5 performace by giving certain index of the result. The file name is profile_[name].json

In [12]:
r.print_profiling(4)

,cat,dur,name,op_name,ph,pid,provider,tid,ts
0,Session,50807,model_loading_uri,NaN,X,1845,NaN,1845,1
1,Session,52344,session_initialization,NaN,X,1845,NaN,1845,50815
2,Node,0,agi.lib.tf.model_repository.spacev_co_train_no...,Gather,X,1845,NaN,1845,103317
3,Node,526,agi.lib.tf.model_repository.spacev_co_train_no...,Gather,X,1845,CPUExecutionProvider,1845,103320
4,Node,0,agi.lib.tf.model_repository.spacev_co_train_no...,Gather,X,1845,NaN,1845,103849
5,Node,0,agi.lib.tf.model_repository.spacev_co_train_no...,Cast,X,1845,NaN,1845,103852
6,Node,2,agi.lib.tf.model_repository.spacev_co_train_no...,Cast,X,1845,CPUExecutionProvider,1845,103853
7,Node,0,agi.lib.tf.model_repository.spacev_co_train_no...,Cast,X,1845,NaN,1845,103858
8,Node,0,agi.lib.tf.model_repository.spacev_co_train_no...,Reshape,X,1845,NaN,1845,103859
9,Node,1,agi.lib.tf.model_repository.spacev_co_train_no...,Reshape,X,1845,CPUExecutionProvider,1845,103861


# Get environment_variables in code_snippet

Given the index from latency.json, output the environment variables.

In [13]:
r.print_environment(3)

,0
LD_LIBRARY_PATH,/perf_test/bin/RelWithDebInfo/cpu_openmp
OMP_WAIT_POLICY,active


# Get codes

Given the index from latency.json, output the codes.

In [15]:
print(r.get_code(3))


import onnxruntime as ort
so = rt.SessionOptions()
so.set_graph_optimization_level(2)
so.enable_sequential_execution = True
so.session_thread_pool_size(0)
session = rt.Session("/mnt/model/test/model.onnx", so)



# netron

In [ ]:
# only workable for notebook in the local server 
import netron
netron.start(model) # 'model.onnx'
from IPython.display import IFrame
IFrame('http://localhost:8080', width=700, height=350)